In [1]:
import pennylane as qml
from pennylane import numpy as np

In [5]:
def is_equivalent(a, b, m):
    """Return a boolean indicating whether the equivalence is satisfied.

    Args:
        a (int): First number to check the equivalence.
        b (int): Second number to check the equivalence.
        m (int): Modulus of the equivalence.

    Returns:
        bool: True if a = b (m), False otherwise.
    """

    ##################
    # YOUR CODE HERE #
    ##################
    return True if a % m == b % m else False



print(f"13 = 8 (3) is {is_equivalent(13, 8, 3)}")
print(f"13 = 7 (6) is {is_equivalent(13, 7, 6)}")


13 = 8 (3) is False
13 = 7 (6) is True


In [6]:
def has_inverse(a, m):
    """Returns a boolean indicating whether a number has an inverse modulo m.

    Args:
        a (int): Number to find the inverse modulus m.
        m (int): Modulus of the equivalence.

    Returns:
        bool: True if c exists (ac = 1 (m)), False otherwise
    """

    ##################
    # YOUR CODE HERE #
    ##################
    for c in range(m):
        if a*c % m == 1 % m:
            return True

    return False

print("(5,15)", has_inverse(5,15))
print("(7,15)", has_inverse(7,15))


(5,15) False
(7,15) True


In [9]:
def nontrivial_square_root(m):
    """Return the first nontrivial square root modulo m.

    Args:
        m (int): modulus for which want to find the nontrivial square root

    Returns:
        int: the first nontrivial square root of m
    """

    ##################
    # YOUR CODE HERE #
    ##################
    for x in range(2, m):
        if x ** 2 % m == 1 % m:
            return x



print(nontrivial_square_root(391))


137


In [2]:
def factorization(N):

    """Return the factors of N.

    Args:
        N (int): number we want to factor.

    Returns:
        array[int]: [p,q] factors of N.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    x = None
    for x_value in range(2, N):
        if x_value ** 2 % N == 1 % N:
            x = x_value
            break

    p = np.gcd(x-1, N)
    q = np.gcd(x+1, N)
    return p, q

N = 391
p, q = factorization(N)
print(f"{N} = {p} x {q}")


391 = 17 x 23


In [6]:
def U():
    qml.SWAP(wires=[2,3])
    qml.SWAP(wires=[1,2])
    qml.SWAP(wires=[0,1])
    for i in range(4):
        qml.PauliX(wires=i)

matrix = qml.matrix(U, wire_order=range(4))()

n_target_wires = 4
target_wires = range(n_target_wires)
n_estimation_wires = 3
estimation_wires = range(4, 4 + n_estimation_wires)


dev = qml.device("default.qubit", shots=1, wires=n_target_wires+n_estimation_wires)

@qml.qnode(dev)
def circuit(matrix):
    """Return a sample after taking a shot at the estimation wires.

    Args:
        matrix (array[complex]): matrix representation of U.

    Returns:
        array[float]: a sample after taking a shot at the estimation wires.
    """

    ##################
    # YOUR CODE HERE #
    ##################

    # CREATE THE INITIAL STATE |0001> ON TARGET WIRES
    qml.PauliX(wires=target_wires[-1])

    # USE THE SUBROUTINE QUANTUM PHASE ESTIMATION
    qml.QuantumPhaseEstimation(
        matrix,
        estimation_wires=estimation_wires,
        target_wires=target_wires
    )

    return qml.sample(wires=estimation_wires)

def get_phase(matrix):
    binary = "".join([str(b) for b in circuit(matrix)])
    return int(binary, 2) / 2 ** n_estimation_wires

for i in range(5):
    print(circuit(matrix))
    print(f"shot {i+1}, phase:",get_phase(matrix))


[1 0 0]
shot 1, phase: 0.5
[0 0 0]
shot 2, phase: 0.0
[1 1 0]
shot 3, phase: 0.25
[0 0 0]
shot 4, phase: 0.25
[1 1 0]
shot 5, phase: 0.0


In [9]:
from fractions import Fraction
def U():
    qml.SWAP(wires=[2,3])
    qml.SWAP(wires=[1,2])
    qml.SWAP(wires=[0,1])
    for i in range(4):
        qml.PauliX(wires=i)

matrix = qml.matrix(U, wire_order=range(4))()

target_wires = range(4)
n_estimation_wires = 3
estimation_wires = range(4, 4 + n_estimation_wires)

def get_period(matrix):
    """Return the period of the state using the already-defined
    get_phase function.

    Args:
        matrix (array[complex]): matrix associated with the operator U

    Returns:
        int: Obtained period of the state.
    """

    shots = 10

    ##################
    # YOUR CODE HERE #
    ##################
    denoms = []
    for i in range(shots):
        x = get_phase(matrix)
        f = Fraction(x).limit_denominator(2 ** n_estimation_wires)
        print(f, f.denominator)
        if x == 0:
            continue
        else:
            denoms.append(f.denominator)

    lcm = 1
    for i in denoms:
        lcm = lcm * i // np.gcd(lcm, i)
    return lcm




print(get_period(matrix))


0 1
1/2 2
1/4 4
0 1
3/4 4
0 1
1/2 2
3/4 4
1/2 2
1/4 4
4
